In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from skimage import io,transform
import torch
from tqdm import tqdm
from torch import nn
import torch.nn.functional as F
import os
import time
from torch.optim import lr_scheduler
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms, utils,models
import copy
import math
from skimage import io, transform
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
!wget http://memorability.csail.mit.edu/lamem.tar.gz
!tar -xf /content/lamem.tar.gz

--2019-12-09 13:24:49--  http://memorability.csail.mit.edu/lamem.tar.gz
Resolving memorability.csail.mit.edu (memorability.csail.mit.edu)... 128.30.195.49
Connecting to memorability.csail.mit.edu (memorability.csail.mit.edu)|128.30.195.49|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2708368436 (2.5G) [application/x-gzip]
Saving to: ‘lamem.tar.gz’

lamem.tar.gz        100%[===================>]   2.52G  32.4MB/s    in 81s     

2019-12-09 13:26:15 (32.0 MB/s) - ‘lamem.tar.gz’ saved [2708368436/2708368436]



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
dataset_train = pd.read_csv("/content/drive/My Drive/image memorability/dataset/train_dataset.csv")
dataset_validation = pd.read_csv("/content/drive/My Drive/image memorability/dataset/validation_dataset.csv")

In [0]:

class AsetheticsDataset(Dataset):
      '''asethitics dataset'''
      def __init__(self,dataframe,root_dir,transform=None):
        """
            Args:
                csv_file (string): Path to the csv file with annotations.
                root_dir (string): Directory with all the images.
                transform (callable, optional): Optional transform to be applied
                    on a sample.
        """
        self.data = dataframe
        #     self.data.rename(columns=columns,inplace=True)
    #     self.data.drop(self.data.columns[[1,2,3,4,5,6,8,9]] , axis=1,inplace=True)
        self.root_dir = root_dir
        self.transform = transform
    
      def __len__(self):
        return len(self.data)
  
      def __getitem__(self,idx):
   
        if torch.is_tensor(idx):
          idx = idx.tolist()
   
        image_name =  os.path.join(self.root_dir,self.data.iloc[idx,0])
        image = io.imread(image_name)
        mem_val = self.data.iloc[idx,1]
#     return_sample={}
        return_sample = {
              'image':image,
              'memorability_score':mem_val 
        }
        if self.transform:
            return_sample = self.transform(return_sample)
    
     
        return return_sample

class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        
        image,mem_val = sample['image'], sample["memorability_score"]
        
        h, w = image.shape[:2]
        
        # if isinstance(self.output_size, int):
        #     if h > w:
        #         new_h, new_w = self.output_size * h / w, self.output_size
        #     else:
        #         new_h, new_w = self.output_size, self.output_size * w / h
        # else:
        #     new_h, new_w = self.output_size

        # new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (self.output_size,self.output_size,3))
        return {'image': img, 'memorability_score': mem_val}
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        
        image, mem_val = sample['image'], sample['memorability_score']
#         print(type(torch.from_numpy(image)))
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
#         print(image.shape)
      
        image = image.transpose((2, 0, 1))
        
        return {'image': torch.from_numpy(image),
                'memorability_score': mem_val}
class Normalize(object):
    def __init__(self,mean,std):
        self.mean=mean
        self.std=std
    def __call__(self,sample):
        image, mem_val = sample["image"], sample["memorability_score"]
        normalized=  (image -self.mean) / self.std
        return {
            "image":normalized,
            "memorability_score" : mem_val
       }

transformed_dataset_train = AsetheticsDataset(dataset_train,root_dir="/content/lamem/images",
                                        transform=transforms.Compose([Rescale(224),ToTensor(),Normalize(0.5,0.5)
                                                          ]))

transformed_dataset_val= AsetheticsDataset(dataset_validation,root_dir="/content/lamem/images",
                                        transform=transforms.Compose([Rescale(224),ToTensor(),Normalize(0.5,0.5)
                                                          ]))

train_dataloader=DataLoader(transformed_dataset_train,batch_size=32,shuffle=True)
validation_dataloader=DataLoader(transformed_dataset_val,batch_size=32,shuffle=True)
dataloaders={
    "train":train_dataloader,
    "val":validation_dataloader
}
dataset_sizes ={
    "train":len(dataset_train),
    "val":len(dataset_validation)
}

In [0]:
#Convolutional layer
class ConvolutionLayer(nn.Module):
    '''
    Args:
    in_channels = the number of channels of input channel (type int)
    output_channels = the number of output feature maps (type int)
    kernel_size = the size of the kernel (type tuple)
    strides = the length of the strides (type int)
    padding = takes two 1) valid, 2) same or some other integer value [default:"valid"]
    
    '''
    def __init__(self,in_channels,out_channels,kernel_size=(3,3),strides=1,padding="valid"):
        super(ConvolutionLayer,self).__init__()
        
        self.in_channels = in_channels
        self.out_channels= out_channels
        self.kernel_size = kernel_size
        self.padding     = padding
        self.strides     = strides
        #layer definition
        self.convolution_layer = self.conv_layer(in_channels,out_channels,kernel_size,strides,padding)
        self.batch_normalize = nn.BatchNorm2d(out_channels)
        self.relu       = nn.ReLU()
        

        
    def forward(self,x):
        x = self.convolution_layer(x)
        x = self.relu(x)
        x = self.batch_normalize(x)
        return x
    
    def conv_layer(self,in_channels,out_channels,kernel_size,strides,padding):
        if padding == "valid":
            padding =0
        elif padding == "same":
            strides = 1
            padding = math.floor(int((kernel_size[0]-1)/2))  
        return nn.Conv2d(in_channels,out_channels,kernel_size,strides,padding)

In [0]:
class LstmCell(nn.Module):
    '''
    Args:
    input_dims : the input dimension (that takes single integer) (type int)
    hidden dims : the dimension of hidden and cell state (type int)
    attach_fc  : to ensure whether fully connected layer should be connected to last rnn cell [default:False]
    '''
    def __init__(self,input_dims,hidden_dims,attach_fc = False):
        super(LstmCell,self).__init__()
        
        self.input_dims = input_dims
        self.hidden_dims = hidden_dims
        self.attach_fc = attach_fc
        
        self.lstm_cell = nn.LSTMCell(input_size = self.input_dims,hidden_size = self.hidden_dims)
        if self.attach_fc == True:
            self.fc = nn.Linear(self.hidden_dims,1)
        
    def forward(self,x,hidden_state,cell_state):
        hidden_output,cell_output = self.lstm_cell(x,(hidden_state,cell_state))
        if self.attach_fc ==True:
            output = self.fc(hidden_output)
            return output
        else:
            return hidden_output,cell_output
        
#     def init_hidden(self, batch_size):
#         hidden = torch.tensor(next(self.parameters()).data.new(batch_size, self.hidden_dims), requires_grad=False)
#         cell = torch.tensor(next(self.parameters()).data.new(batch_size, self.hidden_dims), requires_grad=False)
#         return hidden.zero_(), cell.zero_()
        
        

In [0]:
device

device(type='cuda', index=0)

In [0]:
class VRNet(nn.Module):
    def __init__(self,in_channels,cnn_layer,lstm_cell):
        super(VRNet,self).__init__()
        self.hidden_dims =128
        self.input_dims =128
        self.stage_lstm =lstm_cell(self.input_dims,self.hidden_dims)
        #stage1
        self.stage1_cnn = cnn_layer(in_channels=3,out_channels=32,kernel_size=(3,3),padding="valid",strides=1)
        self.stage1_pool = nn.AvgPool2d(kernel_size=(3,3),stride=2)
        #stage1 inter
        self.stage1_inter_cnn3x3 = cnn_layer(in_channels=32,out_channels=64,kernel_size=(3,3),padding="valid",strides=1)
        self.stage1_inter_cnn1x1 = cnn_layer(in_channels=64,out_channels=128,kernel_size=(1,1),padding="valid",strides=1)
        self.stage1_interpool = nn.AdaptiveAvgPool2d(1)
       

        #stage2
        self.stage2_cnn = cnn_layer(in_channels=32,out_channels=64,kernel_size=(3,3),padding="same",strides=1)
        self.stage2_pool = nn.AvgPool2d(kernel_size=(3,3),stride=2)
        #stage2 inter
        self.stage2_inter_cnn3x3 = cnn_layer(in_channels=64,out_channels=64,kernel_size=(3,3),padding="valid",strides=1)
        self.stage2_inter_cnn1x1 = cnn_layer(in_channels=64,out_channels=128,kernel_size=(1,1))
        self.stage2_interpool = nn.AdaptiveAvgPool2d(1)
        
        #stage3
        self.stage3_cnn = cnn_layer(in_channels=64,out_channels=64,kernel_size=(3,3),padding="valid")
        self.stage3_pool = nn.AvgPool2d(kernel_size=(3,3),stride=1)
        #stage3 inter
        self.stage3_inter_cnn3x3 = cnn_layer(in_channels=64,out_channels=64,kernel_size=(3,3),padding="valid",strides=1)
        self.stage3_inter_cnn1x1 = cnn_layer(in_channels=64,out_channels=128,kernel_size=(1,1))
        self.stage3_interpool = nn.AdaptiveAvgPool2d(1)
        #linear
        self.fc = nn.Linear(self.hidden_dims,1)
        
        
    def forward(self,x,hidden_state,cell_state):
        #stage1
        x = self.stage1_cnn(x)
        x = self.stage1_pool(x)
        #stage1 inter
        x1 = self.stage1_inter_cnn3x3(x)
        x1 = self.stage1_inter_cnn1x1(x1)
        x1  = self.stage1_interpool(x1)
        x1 = x1.squeeze()
        hidden_state,cell_state = self.stage_lstm(x1,hidden_state,cell_state)
        del x1
        
        #stage2
        x = self.stage2_cnn(x)
        x = self.stage2_pool(x)
        #stage2 inter
        x1 = self.stage2_inter_cnn3x3(x)
        x1 = self.stage2_inter_cnn1x1(x1)
        x1 = self.stage2_interpool(x1)

        x1 = x1.squeeze()
        hidden_state,cell_state = self.stage_lstm(x1,hidden_state,cell_state)
        del x1
        #stage3
        x = self.stage3_cnn(x)
        x = self.stage3_pool(x)
        #stage3 inter
        x = self.stage3_inter_cnn3x3(x)
        x = self.stage3_inter_cnn1x1(x)
        x = self.stage3_interpool(x)
        x = x.squeeze()
        hidden_state,cell_state = self.stage_lstm(x,hidden_state,cell_state)
        del x,cell_state
        return self.fc(hidden_state)
        
    def init_hidden(self, batch_size):
        hidden = torch.tensor(next(self.parameters()).data.new(batch_size, self.hidden_dims), requires_grad=False)
        cell = torch.tensor(next(self.parameters()).data.new(batch_size, self.hidden_dims), requires_grad=False)
        return hidden.zero_(), cell.zero_()

In [0]:
model = VRNet(3,ConvolutionLayer,LstmCell).to(device).double()

In [0]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform(m.weight)
    if isinstance(m,nn.Linear):
      torch.nn.init.xavier_uniform(m.weight)
      m.bias.data.fill_(0.01)
        
model.apply(weights_init)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  """


VRNet(
  (stage_lstm): LstmCell(
    (lstm_cell): LSTMCell(128, 128)
  )
  (stage1_cnn): ConvolutionLayer(
    (convolution_layer): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (batch_normalize): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (stage1_pool): AvgPool2d(kernel_size=(3, 3), stride=2, padding=0)
  (stage1_inter_cnn3x3): ConvolutionLayer(
    (convolution_layer): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (batch_normalize): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (stage1_inter_cnn1x1): ConvolutionLayer(
    (convolution_layer): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    (batch_normalize): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (stage1_interpool): AdaptiveAvgPool2d(output_size=1)
  (stage2_cnn): ConvolutionLayer(
    (convolution_layer): Conv2d(32, 64, kernel

In [0]:
# def init_weights(m):
#     if type(m) == nn.Linear:
#         torch.nn.init.xavier_uniform(m.weight)
#         m.bias.data.fill_(0.01)

# net = nn.Sequential(nn.Linear(2, 2), nn.Linear(2, 2))
# net.apply(init_weights)

In [0]:
t = torch.rand([32,3,224,224]).to(device).double()

In [0]:
hidden_state ,cell_state = model.init_hidden(16)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [0]:
# hidden_state = hidden_state.to(device)
# cell_state = cell_state.to(device)

In [0]:
with torch.no_grad():
  x =model(t,hidden_state,cell_state)
  print(x.shape)

torch.Size([32, 1])


In [0]:
hidden_state.shape

torch.Size([32, 128])

In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()
    running_loss_history = []
    val_running_loss_history=[]
    orignal_model=None
    best_model_wts = copy.deepcopy(model.state_dict())
    low_loss = np.inf

    for epoch in range(num_epochs):
       
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for batched_data in tqdm(dataloaders[phase]):
                
                inputs=batched_data["image"]
                inputs = inputs.to(device)
                hidden_state ,cell_state = model.init_hidden(inputs.size(0))
                if inputs.size(0) == 16:
                   print(hidden_state.shape)
                labels=batched_data["memorability_score"]
                labels=labels.view(-1,1).double()
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs,hidden_state,cell_state)
                    loss = criterion(outputs, labels)
                    print("  batch loss:    ",loss.item())
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            if phase=="train":
              running_loss_history.append(epoch_loss)
            else:
              val_running_loss_history.append(epoch_loss)
            

            print('{} Loss: {:.4f}'.format(
                phase, epoch_loss))
            

            # deep copy the model
            if phase == 'val' and epoch_loss < low_loss:
                print("saving best model......")
                low_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                #torch.save(best_model_wts,"/content/drive/My Drive/image memorability/saved models/resnet50_weights(2).pt")

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    # load best model weights
    original_model =copy.deepcopy(model)
    model.load_state_dict(best_model_wts)
    return model,original_model,running_loss_history,val_running_loss_history

In [0]:

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [0]:
best_model,model,running_loss_history,val_running_loss_history = train_model(best_model,criterion,optimizer,exp_lr_scheduler,5)
# torch.save(best_model.state_dict(),"/content/drive/My Drive/image memorability/saved models/Vnet_weights(2).pt")





  0%|          | 0/1469 [00:00<?, ?it/s]

Epoch 0/4
----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  batch loss:     0.00938184464701455





  0%|          | 1/1469 [00:02<50:16,  2.06s/it]

...

  batch loss:     0.00864206193406938





 99%|█████████▉| 366/368 [10:22<00:03,  1.69s/it]

  batch loss:     0.016668177059065575





100%|█████████▉| 367/368 [10:24<00:01,  1.76s/it]


100%|██████████| 368/368 [10:24<00:00,  1.28s/it]




  batch loss:     0.014371115620943512
  batch loss:     0.011304371832087464
val Loss: 0.0132

Training complete in 310m 19s
